In [555]:
import pandas as pd
import imageio 
import re
import numpy as np
import os
from pathlib import Path
import tensorflow as tf
from sklearn.utils import shuffle
from skimage import color



np.set_printoptions(threshold=np.inf)
from matplotlib import pyplot as plt

class primusSequence:

    def __init__(self, folder, package):
        self.image = imageio.imread("./data/" + package + "/" + folder + "/" + folder + ".png")
        self.semantic = open("./data/" + package + "/" + folder + "/" + folder + ".semantic").read()
        self.labels = re.findall(r'note\-(.+?)\_', self.semantic)
        
    def flattened_image(self):
        im = self.image
        return(im.flatten())
                                    
        
def init_objs(dataset):
    
    if dataset == 'train':
        ROOT = './data/package_aa/'
        package = 'package_aa'
    elif dataset == 'validate':
        ROOT = './data/package_ab/'
        package = 'package_ab'

    folders = [directory for directory in os.listdir(ROOT) if os.path.isdir(ROOT+directory)]
    objs = []
    
    for f in folders:
        objs.append(primusSequence(f, package))
        
    return(objs)

In [76]:
objs = init_objs('train')

In [146]:
train_labels = ['a', 'b'] * 21797

In [148]:
train_images = np.array(train_images)

/Users/George/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [790]:
def map_pitch(pitch):
    
    m = {"a": 0,
         "h": 1,
         "c1": 2,
         "d1": 3,
         "e1": 4,
         "f1": 5,
         "g1": 6,
         "a1": 0,
         "h1": 1,
         "c2": 2,
         "d2": 3,
         "e2": 4,
         "f2": 5,
         "g2": 6,
         "a2": 0,
         "h2": 1,
         "c3": 2,
         "other": 7}
    
    return(m[pitch])

class ViennaNote:
    
    def __init__(self, fn, typ):
        attr = fn.split('-')
        
        if typ == 'test':
            ROOT = './data/originals-test/'
        elif typ == 'train':
            ROOT = './data/transformations-resized/'
        else:
            sys.exit()
            
        self.type = attr[0]
        self.time = attr[1]
        self.pitch = attr[2]
        imfile = imageio.imread(ROOT + fn)
        self.image = color.rgb2gray(imfile) / 255
        
    def resize_image(self, width, height):
        pass
    
    
class InputData:
    
    ROOT = './data/transformations-resized/' 
    
    def __init__(self, path = ROOT):
        self.files = [file for file in os.listdir(path) if os.path.isfile(path + file) and file[0:4] == 'note']
        self.objs = [ViennaNote(f, 'train') for f in self.files]
                
    def training_images(self, path = ROOT):
        images = [o.image for o in self.objs]
        return(np.array(images))
    
    def training_labels(self, path = ROOT):
        notes = [o.pitch for o in self.objs]
        labels = [map_pitch(pitch) for pitch in notes]
        return(np.array(labels))

In [791]:
input_data = InputData()

In [614]:
x = input_data.training_images()

In [442]:
x = np.array(x)

In [792]:
input_data.training_images().shape

(39264, 50, 30)

In [793]:
train_images = input_data.training_images().reshape(39264, 50, 30, 1)
train_labels = input_data.training_labels()

In [794]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(3, 5, input_shape=(50, 30, 1)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Flatten(input_shape = (50, 30, 1)),
    tf.keras.layers.Dense(800, activation=tf.nn.relu),
    tf.keras.layers.Dense(160, activation=tf.nn.softmax),
])

In [795]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [801]:
train_images, train_labels = shuffle(train_images, train_labels)

model.fit(train_images, 
          train_labels,
          validation_split=0.3,
          epochs=5)

Epoch 1/5
859/859 [==============================] - 22s 26ms/step - loss: 0.0177 - accuracy: 0.9951 - val_loss: 0.0294 - val_accuracy: 0.9912
Epoch 2/5
859/859 [==============================] - 28s 33ms/step - loss: 0.0093 - accuracy: 0.9970 - val_loss: 0.0155 - val_accuracy: 0.9947
Epoch 3/5
859/859 [==============================] - 29s 34ms/step - loss: 0.0086 - accuracy: 0.9975 - val_loss: 0.0084 - val_accuracy: 0.9975
Epoch 4/5
859/859 [==============================] - 27s 31ms/step - loss: 0.0150 - accuracy: 0.9956 - val_loss: 0.0198 - val_accuracy: 0.9948
Epoch 5/5
859/859 [==============================] - 28s 33ms/step - loss: 0.0055 - accuracy: 0.9981 - val_loss: 0.0081 - val_accuracy: 0.9975


In [780]:
model.summary()

Model: "sequential_110"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 46, 26, 3)         78        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 23, 13, 3)         0         
_________________________________________________________________
flatten_99 (Flatten)         (None, 897)               0         
_________________________________________________________________
dense_228 (Dense)            (None, 800)               718400    
_________________________________________________________________
dense_229 (Dense)            (None, 160)               128160    
Total params: 846,638
Trainable params: 846,638
Non-trainable params: 0
_________________________________________________________________


In [320]:
shuff

In [597]:
train_labels.shape

(3272,)

In [781]:
model.predict_classes(test_data)

/Users/George/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([2, 4, 0, 4, 0, 0, 2, 4, 4, 2, 2, 0, 2, 0, 0, 0])

In [657]:
test_image = imageio.imread('./data/originals-resized/note-eighth-f1-3159.png')

In [658]:
test_image = color.rgb2gray(test_image) / 255

In [679]:
test = input_data.training_images()[1:5]

In [680]:
test.shape

(4, 50, 30)

In [763]:
class ViennaNote:
    
    def __init__(self, fn, typ):
        attr = fn.split('-')
        
        if typ == 'test':
            ROOT = './data/originals-test/'
        elif typ == 'train':
            ROOT = './data/originals-resized/'
        else:
            sys.exit()
            
        self.type = attr[0]
        self.time = attr[1]
        self.pitch = attr[2]
        imfile = imageio.imread('./data/originals-test/' + fn)
        self.image = color.rgb2gray(imfile) / 255
        
    def resize_image(self, width, height):
        pass
    

class TestData:
    
    ROOT = './data/originals-test/' 
    
    def __init__(self, path = ROOT):
        self.files = [file for file in os.listdir(path) if os.path.isfile(path + file) and file[0:4] == 'note']
        self.objs = [ViennaNote(f, 'test') for f in self.files]
                
    def training_images(self, path = ROOT):
        images = [o.image for o in self.objs]
        return(np.array(images))
    
    def training_labels(self, path = ROOT):
        notes = [o.pitch for o in self.objs]
        labels = [map_pitch(pitch) for pitch in notes]
        return(np.array(labels))

In [798]:
test_data = TestData().training_images().reshape(16, 50, 30, 1)
labs = TestData().training_labels()

In [799]:
labs

array([2, 4, 0, 4, 0, 0, 2, 4, 3, 2, 3, 0, 2, 0, 0, 0])

In [800]:
model.predict_classes(test_data)

array([2, 4, 0, 4, 0, 0, 2, 4, 3, 2, 3, 0, 2, 0, 0, 0])